In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe pose model
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Helper function to calculate angle between 3 points
def calculate_angle(a, b, c):
    """
    Calculates the angle at point 'b' given three coordinates:
    a (hip), b (knee), c (ankle) or similar joints.
    """
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    # Vector calculations
    ba = a - b
    bc = c - b

    # Calculate angle in radians and convert to degrees
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Open webcam
cap = cv2.VideoCapture(0)

# Variables for squat counting
counter = 0
stage = None  # "up" or "down"

# Depth bar configuration
bar_max_width = 300
ideal_knee_angle = 90  # Ideal squat depth

# Start pose detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the BGR frame to RGB for MediaPipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Perform pose detection
        results = pose.process(image)

        # Convert image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for left leg joints
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate knee angle
            knee_angle = calculate_angle(hip, knee, ankle)

            # Squat counter logic
            if knee_angle > 160:
                stage = "up"
            if knee_angle < 100 and stage == 'up':
                stage = "down"
                counter += 1

            # ---- Form Depth Feedback ----
            # Calculate how close the knee angle is to ideal depth
            form_score = max(0, min(100, 100 - abs(knee_angle - ideal_knee_angle)))
            bar_width = int((form_score / 100) * bar_max_width)

            # Choose bar color based on form score
            if form_score > 80:
                bar_color = (0, 255, 0)  # Green for good form
                feedback = "Perfect Depth ✅"
            elif form_score > 50:
                bar_color = (0, 165, 255)  # Orange for medium
                feedback = "Go Lower ⬇️"
            else:
                bar_color = (0, 0, 255)  # Red for bad form
                feedback = "Too Shallow ⚠️"

            # Draw form bar
            cv2.rectangle(image, (50, 50), (50 + bar_max_width, 80), (200, 200, 200), 2)
            cv2.rectangle(image, (50, 50), (50 + bar_width, 80), bar_color, -1)

            # Display feedback text
            cv2.putText(image, feedback, (50, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, bar_color, 2)

        except:
            pass

        # Display squat count
        cv2.putText(image, f"Squats: {counter}", (10, 450),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)

        # Draw pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Show image
        cv2.imshow('Squat Counter with Depth Feedback', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()